In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaModel

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, precision_score, recall_score
from tqdm.notebook import trange, tqdm
import numpy as np

In [4]:
# Load the dataset from a CSV file
df = pd.read_csv('outdata_M.csv')

In [5]:
df

,NOTES,EVENT_TYPE
0,5 January: Beheading of 5 citizens in Douaouda...,2
1,Two citizens were beheaded in Hassasna.,2
2,Two citizens were killed in a raid on the vill...,2
3,4 January: 16 citizens were murdered in the vi...,2
4,5 January: Killing of 18 citizens in the Olivi...,2
...,...,...
65528,OLF and Borana ethnic militia attack civilians...,2
65529,Ethnic raiders and OLF attack Degodia Somali e...,2
65530,OLF and Borana ethnic militia attack civilians...,2
65531,University students protest over killing in Wa...,3


In [6]:
df.shape

(65533, 2)

# **Text Preprocessing: **
Clean the text data by removing stop words, special characters, punctuation, etc.

In [7]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer



# Lowercase text
df["NOTES"] = df["NOTES"].str.lower()

# Remove punctuation
df["NOTES"] = df["NOTES"].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)) if isinstance(x, str) else '')


# Remove stop words
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))
df["NOTES"] = df["NOTES"].apply(lambda x: " ".join([word for word in x.split() if word not in stop_words]))

# Remove numbers
df["NOTES"] = df["NOTES"].apply(lambda x: re.sub(r'\d', '', str(x)))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
df

,NOTES,EVENT_TYPE
0,january beheading citizens douaouda tipaza,2
1,two citizens beheaded hassasna,2
2,two citizens killed raid village hassi el abd ...,2
3,january citizens murdered village benachour ...,2
4,january killing citizens oliviers district d...,2
...,...,...
65528,olf borana ethnic militia attack civilians mas...,2
65529,ethnic raiders olf attack degodia somali ethni...,2
65530,olf borana ethnic militia attack civilians mas...,2
65531,university students protest killing wajir dist...,3


In [9]:
df.isnull().sum()

NOTES         0
EVENT_TYPE    0
dtype: int64

In [10]:
df.dropna(axis=0, how="any", thresh=None, subset=None, inplace=False).shape

(65533, 2)

In [11]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [12]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe['NOTES']
        self.targets = dataframe['EVENT_TYPE']
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.long)
        }

In [13]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 128
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
EPOCHS = 10
LEARNING_RATE = 0.01

In [14]:
# # Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [15]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=False)

In [16]:
training_set = CustomDataset(train_df, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_df, tokenizer, MAX_LEN)

In [17]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

In [18]:
training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [19]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained('roberta-base')
        self.l2 = torch.nn.Dropout(0.5)
        self.l3 = torch.nn.Linear(768, 6)
    
    def forward(self, ids, mask, token_type_ids):
        # breakpoint()
        # _, output_1= self.l1(ids, attention_mask=mask, token_type_ids=token_type_ids)
        output_1 = self.l1(ids, attention_mask=mask, token_type_ids=token_type_ids)
        output_2 = self.l2(output_1.pooler_output)
        output = self.l3(output_2)
        return output

model = RobertaClass()
model.to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), e

In [20]:
def loss_fn(outputs, targets):
    return torch.nn.CrossEntropyLoss()(outputs, targets)

In [21]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [22]:
def train(epoch):
    num_of_batches = int(len(train_df)/training_loader.batch_size)
    model.train()
    running_loss = 0.0    
    counter = 0

    for data in tqdm(training_loader, total=num_of_batches, desc="Epoch No: " + str(epoch+1), colour="blue"):
        counter += 1

        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets-1)
        tqdm.write(f"Training Iteration {counter}, Loss: {loss.item()}")
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    return running_loss/counter

In [23]:
def get_class(outputs):
    scores = torch.tensor(outputs)
    probs = torch.nn.functional.softmax(scores, dim=1)
    print(probs)

    _, predictions = torch.max(probs, dim=1)
    predictions += 1
    return predictions.tolist()

In [24]:
def validation(loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    running_loss = 0.0    
    counter = 0
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            counter += 1
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(outputs.cpu().detach().numpy().tolist())
            loss = loss_fn(outputs, targets-1)
            running_loss += loss.item()
    loss = running_loss/counter
    fin_outputs = get_class(fin_outputs)
    return fin_outputs, fin_targets, loss

In [25]:
for epoch in trange(EPOCHS, desc='Total Epochs: ', colour="green"):
    train_epoch_loss = train(epoch)
    #print(f"Test Average Recall: {test_average_recall:.4f}")
    outputs_train, targets_train, train_epoch_loss = validation(training_loader)

    #  Testing started from here
    #  evaluate on training data
    train_accuracy = accuracy_score(targets_train, outputs_train)
    print(f"Training loss for epcoh {epoch+1}: {train_epoch_loss}")
    print(f"Training accuracy for epoch {epoch+1}: {train_accuracy}")


    # evaluate on testing data
    outputs_test, targets_test, valid_epoch_loss = validation(testing_loader)
    test_accuracy = accuracy_score(targets_test, outputs_test)
    print(f"Validation loss for epcoh {epoch+1}: {valid_epoch_loss}")
    print(f"Test accuracy for epoch {epoch+1}: {test_accuracy}")

Total Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch No: 1:   0%|          | 0/6553 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Streaming output truncated to the last 5000 lines.
Training Iteration 1564, Loss: 5.476740837097168
Training Iteration 1565, Loss: 4.142246246337891
Training Iteration 1566, Loss: 4.291141510009766
Training Iteration 1567, Loss: 8.08358383178711
Training Iteration 1568, Loss: 2.3080925941467285
Training Iteration 1569, Loss: 6.173572540283203
Training Iteration 1570, Loss: 2.9670050144195557
Training Iteration 1571, Loss: 3.9319076538085938
Training Iteration 1572, Loss: 7.4030561447143555
Training Iteration 1573, Loss: 3.511410713195801
Training Iteration 1574, Loss: 7.062469959259033
Training Iteration 1575, Loss: 2.5509140491485596
Training Iteration 1576, Loss: 3.1586897373199463
Training Iteration 1577, Loss: 8.117525100708008
Training Iteration 1578, Loss: 6.536108016967773
Training Iteration 1579, Loss: 4.543871879577637
Training Iteration 1580, Loss: 3.7554030418395996
Training Iteration 1581, Loss: 4.950090408325195
Training Iteration 1582, Loss: 2.186807632446289
Training Ite

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


...

Training Iteration 1, Loss: 4.770269393920898


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Streaming output truncated to the last 5000 lines.
Training Iteration 1564, Loss: 3.7350282669067383
Training Iteration 1565, Loss: 4.820809841156006
Training Iteration 1566, Loss: 3.1551878452301025
Training Iteration 1567, Loss: 3.234713315963745
Training Iteration 1568, Loss: 4.789275169372559
Training Iteration 1569, Loss: 4.314996242523193
Training Iteration 1570, Loss: 7.6658616065979
Training Iteration 1571, Loss: 3.4667515754699707
Training Iteration 1572, Loss: 5.563244819641113
Training Iteration 1573, Loss: 2.4524025917053223
Training Iteration 1574, Loss: 8.049346923828125
Training Iteration 1575, Loss: 3.8093323707580566
Training Iteration 1576, Loss: 6.28852653503418
Training Iteration 1577, Loss: 3.010226249694824
Training Iteration 1578, Loss: 3.834747552871704
Training Iteration 1579, Loss: 2.357412815093994
Training Iteration 1580, Loss: 6.966704845428467
Training Iteration 1581, Loss: 5.9711809158325195
Training Iteration 1582, Loss: 2.700460910797119
Training Iterat

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


tensor([[1.9933e-01, 7.7769e-01, 7.6592e-03, 1.5229e-02, 9.7458e-05, 2.1226e-12],
        [1.9933e-01, 7.7769e-01, 7.6592e-03, 1.5229e-02, 9.7458e-05, 2.1226e-12],
        [1.9933e-01, 7.7769e-01, 7.6592e-03, 1.5229e-02, 9.7458e-05, 2.1226e-12],
        ...,
        [1.9933e-01, 7.7769e-01, 7.6592e-03, 1.5229e-02, 9.7458e-05, 2.1226e-12],
        [1.9933e-01, 7.7769e-01, 7.6592e-03, 1.5229e-02, 9.7458e-05, 2.1226e-12],
        [1.9933e-01, 7.7769e-01, 7.6592e-03, 1.5229e-02, 9.7458e-05, 2.1226e-12]])
Validation loss for epcoh 10: 3.6397840332301237
Test accuracy for epoch 10: 0.2780956740672923
